# Exemplo 3: Saúde - Sinais Vitais de Pacientes

Este notebook demonstra o monitoramento de sinais vitais de pacientes em tempo real usando **Apache Spark Streaming** e **Kafka**.

**Cenário**: Detectar condições críticas (ex: SpO2 baixo ou batimentos cardíacos elevados) em pacientes monitorados.

## 1. Configuração do Ambiente
Instalação do Java, Spark, Kafka e bibliotecas Python.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q  https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!wget -q https://downloads.apache.org/kafka/3.6.1/kafka_2.13-3.6.1.tgz
!tar xf kafka_2.13-3.6.1.tgz
!pip install -q findspark pyspark kafka-python

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()

## 2. Iniciar Kafka
Serviços em background.

In [ ]:
%%bash
cd kafka_2.13-3.6.1
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
sleep 5
bin/kafka-server-start.sh -daemon config/server.properties
sleep 5
bin/kafka-topics.sh --create --topic vital-signs --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

## 3. Simulador de Sensores Médicos
Emitindo dados de Batimentos (BPM), Pressão Arterial (BP) e Oxigenação (SpO2).

In [ ]:
import time
import json
import random
from kafka import KafkaProducer
import threading

def generate_vitals():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))
    patient_ids = [101, 102, 103, 104, 105]
    
    try:
        for _ in range(200):
            for pid in patient_ids:
                data = {
                    'patient_id': pid,
                    'timestamp': time.time(),
                    'heart_rate': random.randint(60, 140),
                    'systolic_bp': random.randint(100, 160),
                    'diastolic_bp': random.randint(60, 100),
                    'spo2': random.randint(85, 100)
                }
                producer.send('vital-signs', value=data)
            time.sleep(1)
    except Exception as e:
        print(e)
    finally:
        producer.close()

thread = threading.Thread(target=generate_vitals)
thread.start()

## 4. Monitoramento Crítico com Spark Streaming
Regra de alerta: SpO2 < 90 OU Heart Rate > 120.

In [ ]:
%%writefile kafka_consumer.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, when
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

spark = SparkSession.builder \
    .appName("HealthcareMonitor") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

schema = StructType([
    StructField("patient_id", IntegerType()),
    StructField("timestamp", FloatType()),
    StructField("heart_rate", IntegerType()),
    StructField("systolic_bp", IntegerType()),
    StructField("diastolic_bp", IntegerType()),
    StructField("spo2", IntegerType())
])

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "vital-signs") \
    .load()

vitals = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Identificar condições críticas
alerts = vitals.withColumn("status", 
    when((col("spo2") < 90) | (col("heart_rate") > 120), "CRITICAL")
    .otherwise("NORMAL")
)

critical_patients = alerts.filter(col("status") == "CRITICAL")

query = critical_patients.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()


query.awaitTermination()

In [ ]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 kafka_consumer.py